In [2]:
# GENERAL
from fastapi import FastAPI, File, Form, UploadFile, HTTPException, Body
from typing import Dict, Any
from typing import List, Annotated
import asyncio
import random
import tempfile
import shutil
import os
import fitz
import io
import base64
import datetime
import hashlib
import time
import anyio
import requests
import json
import simple_salesforce
from PIL import Image

# URLLIB3
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# AZURE AI DOCUMENT INTELLIGENCE
from azure.core.credentials import AzureKeyCredential
from azure.ai.documentintelligence import DocumentIntelligenceClient

# LOAD ENV VARIABLES
from dotenv import load_dotenv
load_dotenv()

# CUSTOM UTILS
from customutils import *

In [3]:
def _conSF():
    # GET TOKEN IF NO TOKEN YET
    if 'SALESFORCE_ACCESS_TOKEN' not in os.environ or 'SALESFORCE_INSTANCE_URL' not in os.environ:
        # GET TOKEN
        import requests
        url = os.getenv('SFDC_URL')
        payload = {}
        headers = {'Authorization': os.getenv('SFDC_AUTH'),
                'Cookie': os.getenv('SFDC_COOKIE')}
        response = requests.request("POST", url, headers=headers, data=payload)
        os.environ['SALESFORCE_ACCESS_TOKEN'] = response.json()['access_token']
        os.environ['SALESFORCE_INSTANCE_URL'] = response.json()['instance_url']
    # MAIN
    from simple_salesforce import Salesforce
    sf = Salesforce(instance_url=os.environ['SALESFORCE_INSTANCE_URL'], session_id=os.environ['SALESFORCE_ACCESS_TOKEN'])
    return sf

In [4]:
import pandas as pd

def queryAllSf(tableName):
    # CONNECT TO SALESFORCE USING YOUR EXISTING FUNCTION
    sfConn = _conSF()

    # DYNAMICALLY ACCESS THE OBJECT (TABLE) AND DESCRIBE IT
    # THIS FETCHES METADATA INCLUDING ALL FIELD NAMES
    sfObj = getattr(sfConn, tableName)
    objDesc = sfObj.describe()

    # EXTRACT ALL FIELD NAMES FROM THE METADATA
    fieldList = [field['name'] for field in objDesc['fields']]

    # CONSTRUCT THE SOQL QUERY
    # JOIN ALL FIELD NAMES WITH COMMA
    soqlQuery = "SELECT " + ",".join(fieldList) + " FROM " + tableName

    # EXECUTE THE QUERY
    # USE QUERY_ALL TO RETRIEVE ALL RECORDS AUTOMATICALLY HANDLIN PAGINATION
    queryResult = sfConn.query_all(soqlQuery)

    # CONVERT TO PANDAS DATAFRAME
    if queryResult['totalSize'] > 0:
        sfDf = pd.DataFrame(queryResult['records'])
        
        # REMOVE THE 'ATTRIBUTES' METADATA COLUMN
        if 'attributes' in sfDf.columns:
            sfDf = sfDf.drop(columns=['attributes'])
    else:
        # RETURN EMPTY DATAFRAME WITH CORRECT COLUMNS IF NO DATA FOUND
        sfDf = pd.DataFrame(columns=fieldList)

    return sfDf

In [11]:
dfFUNC = queryAllSf('function__c')
dfFUNC = dfFUNC[dfFUNC['IsDeleted'] == False]
dfFUNC = dfFUNC[['Name','Industry_Cluster__c']].drop_duplicates().reset_index(drop=True)

dfIC = queryAllSf('Industry_Cluster__c')
dfIC = dfIC[['Id','Business_Line_Name_Formula__c']].drop_duplicates()
dfIC.columns = ['Industry_Cluster__c','Business_Line_Name_Formula__c']

dfFUNC = dfFUNC.merge(dfIC, on='Industry_Cluster__c', how='left')
dfFUNC = dfFUNC[['Business_Line_Name_Formula__c','Name']].drop_duplicates().reset_index(drop=True)

dfFUNC.to_excel('ZMAP_Functions.xlsx', index=False)

In [10]:
dfAPPL = queryAllSf('application__c')
dfAPPL = dfAPPL[dfAPPL['IsDeleted'] == False]
dfAPPL = dfAPPL[['Name','Industry_Cluster__c']].drop_duplicates().reset_index(drop=True)

dfIC = queryAllSf('Industry_Cluster__c')
dfIC = dfIC[['Id','Business_Line_Name_Formula__c']].drop_duplicates()
dfIC.columns = ['Industry_Cluster__c','Business_Line_Name_Formula__c']  

dfAPPL = dfAPPL.merge(dfIC, on='Industry_Cluster__c', how='left')
dfAPPL = dfAPPL[['Business_Line_Name_Formula__c','Name']].drop_duplicates().reset_index(drop=True)

dfAPPL.to_excel('ZMAP_Applications.xlsx', index=False)